In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from xml.etree.ElementTree import *
import glob
import os
import shutil

In [2]:
# Load imx.file
filelist=glob.glob('*00.imx')
tree = parse(filelist[0])
root = tree.getroot()
cpname = filelist[0]+"_edit.imx"
shutil.copyfile(filelist[0], cpname)

'Lot170502_K14rtTA-TetOH2BEGFP-Fucci596-0203_MeasurementPoints+Surfaces00.imx_edit.imx'

In [3]:
# Extract vertices of surfaces from imx file
for bps in root.iter("bpSurfaces"):
    #print bps.tag
    for name in bps.iter("name"):
        surface_name=name.text
        #print (surface_name)
        
        k=0
        for surface in bps.findall("surface"):
            
            i=0
            for vtc in surface.iter("vertices"):
                i+=1
            #print e.tag
            nlist = i;
            #print ("# of vertices tag is")
            #print (nlist)
            i=0;
            vlist=[];
        
            for vtc in surface.iter("vertices"):
                tmp=vtc.text
                tmp2=tmp.replace('\"','')
                A=tmp2.split()
                vlist.extend(A)
                #tmp.split
                i+=1
            vlist_np=np.array(vlist)
            #print (vlist_np.dtype)
            #print (len(vlist))
        
            num_vtx=int(len(vlist)/6)
            #print ("# of vertices is")
            #print(num_vtx)
            vlist_np=vlist_np.reshape(num_vtx,6)
            #print ("# of vertices")
            #print (len(vlist_np))
            vlist_np_fl=vlist_np.astype(np.float64)
            
            for time in surface.findall("timeIndex"):
                time_num = time.text
            #filename="vertices_"+surface_name+"_time"+str(time_num)+".txt"
            filename="vertices_surface_time"+str(time_num)+".txt"
            np.savetxt(filename,vlist_np_fl,delimiter=",")
        
            k+=1

In [4]:
#select basal membrane of interest by conditining normal vector and xyz coordinates of vertices, then save vertex information to "vertices_surface_time"+str(time_num)+"_subset"+".txt"
for bps in root.iter("bpSurfaces"):
    
    #print bps.tag
    for name in bps.iter("name"):
        #surface_name=name.text
        for surface in bps.findall("surface"):
            for time in surface.findall("timeIndex"):
                time_num = time.text
                #print(time_num)
        
            #filename = "vertices_"+surface_name+"_time"+str(time_num)+".txt"
            filename = "vertices_surface_time"+str(time_num)+".txt"
            vlist_np_fl=np.loadtxt(filename,delimiter=",")
            
            #select basal membrane by thresholding normal vector. This condition depends on the data
            #vlist_np_fl_edit = vlist_np_fl[vlist_np_fl[:,5]>0.1]
            
            vlist_np_fl_edit = vlist_np_fl[vlist_np_fl[:,4]>-0.1]
            vlist_np_fl_edit = vlist_np_fl_edit[vlist_np_fl_edit[:,2]>-1459]
            vlist_np_fl_edit = vlist_np_fl_edit[vlist_np_fl_edit[:,2]<(-1318.8)]
            vlist_np_fl_edit = vlist_np_fl_edit[vlist_np_fl_edit[:,0]>0]
            vlist_np_fl_edit = vlist_np_fl_edit[vlist_np_fl_edit[:,0]<338]
            vlist_np_fl_edit = vlist_np_fl_edit[vlist_np_fl_edit[:,1]>105]
            
            #print ("# of vertices is")
            #print(vlist_np_fl_edit.shape[0])
            #filename = "vertices_"+surface_name+"_time"+str(time_num)+"_subset"+".txt"
            filename = "vertices_surface_time"+str(time_num)+"_subset"+".txt"
            np.savetxt(filename,vlist_np_fl_edit,delimiter=",")

In [6]:
# Extract measurement points
for bpsa in root.findall("bpSurfaceApplication"):
    for bpcg in bpsa.findall("bpComponentGroup"):
        #print bpcg.tag
        for bpsc in bpcg.findall("bpSurfaceComponent"):
            #print bpsc.tag
            for bpcg2 in bpsc.findall("bpComponentGroup"):
                #print bpcg2.tag
                for bpsc2 in bpcg2.findall("bpSurfaceComponent"):
                    #print bpsc2.tag
                    for name in bpsc2.findall("name"):
                        bpsc_name = name.text
                        #print (bpsc_name)
                        #if ((bpsc_name.startswith("Frame"))==True):
                            #print bpsc_name
                        #counting # of spindles in one Frame
                        i=0
                        for bpmp in bpsc2.findall("bpMeasurementPoints"):
                            for bpsc3 in bpmp.findall("bpSurfaceComponent"):
                                for name2 in bpsc3.findall("name"):
                                    #print name2.text
                                    i+=1
                        nlist = i;
                        #print ("# of bpMeasurementPoints tag")
                        #print (nlist)
                                    

                        vlist=[[0 for i in range(5)] for j in range(nlist*2+1)]
                        #print vlist
                        vlist[0]=["mitotic.spindle","t","x","y","z"]
                        name_sp=[0 for j in range(nlist)]
                        #print name_sp
                        k=1;
                        
                        for bpmp in bpsc2.findall("bpMeasurementPoints"):
                            for bpsc3 in bpmp.findall("bpSurfaceComponent"):
                                for name2 in bpsc3.findall("name"):
                                    sp_name=name2.text
                                    #print (sp_name)
                                    A=sp_name.split()

                            j=0;
                            for mpoint in bpmp.findall("Point"):
                                tmp=mpoint.attrib
                                xyz = mpoint.get('Position')
                                T = mpoint.get('TimeIndex')
                                XYZ=xyz.split()
                                AorB = mpoint.get('Name')
                                vlist[k*2-1+j][0]=A[2]+"_"+AorB;
                                vlist[k*2-1+j][1]=T;
                                vlist[k*2-1+j][2:5]=XYZ;
                                #print (T,xyz);
                                j+=1;
                            name_sp[k-1]=A;
                            k+=1

                        #print vlist

                        filename = 'mitotic_spindles_'+bpsc_name+'.txt'
                        f = open(filename, 'w')
                        for x in vlist:
                            for y in x:
                                f.write(str(y)+" ")
                            f.write("\n")
                        f.close()

In [7]:
# Culculation of angles between surface normal of basal membrane and spindle        
for bpsa in root.findall("bpSurfaceApplication"):
    for bpcg in bpsa.findall("bpComponentGroup"):
        #print bpcg.tag
        for bpsc in bpcg.findall("bpSurfaceComponent"):
            #print bpsc.tag
            for bpcg2 in bpsc.findall("bpComponentGroup"):
                #print bpcg2.tag
                for bpsc2 in bpcg2.findall("bpSurfaceComponent"):
                    #print bpsc2.tag
                    for name in bpsc2.findall("name"):
                        bpsc_name = name.text
                        #print (bpsc_name)
                        filename = "mitotic_spindles_"+bpsc_name+".txt"
                        #spindle=np.loadtxt(filename,skiprows=1,usecols=(1,2,3,4))
                        spindle0=pd.read_table(filename, sep=' ', usecols=(0,1,2,3,4))
                        #print(spindle0)    
                        
                        #print len(spindle)
                        num_spindle=int(len(spindle0)/2);
                        #print  num_spindle
                        #result=np.zeros((num_spindle,15))
                        result=pd.DataFrame(np.zeros((num_spindle,15)))
                        
                        for i in range(num_spindle):
                            #print (i)
                            
                            xyz_A=spindle0.iloc[2*i,2:5]
                            xyz_B=spindle0.iloc[2*i+1,2:5]
                            time_num=spindle0.iloc[2*i,1]
                            spindle_name=spindle0.iloc[2*i,0]
                            spindle_name=spindle_name.split("_")[0]
                            #print(time_num)
                            #print(spindle_name)
                            
                            #filename="vertices_Surface_"+bpsc_name+"_time"+str(time_num)+"_subset"+".txt"
                            filename="vertices_surface_time"+str(time_num)+"_subset"+".txt"
                            vlist_np_fl_edit=np.loadtxt(filename,delimiter=",")
                            
                            #calculation of center of mass of a spindle
                            center=(xyz_A+xyz_B)/2.0;
                            #calculation of a unit vector of a spindle
                            vector=xyz_B-xyz_A
                            unit_vector=vector/np.linalg.norm(vector)

                            #preparation of ndarry
                            length=np.zeros((len(vlist_np_fl_edit),1));
                            sp_dum=np.ones((len(vlist_np_fl_edit),3))
                            #dummy ndarray for the position of spindle, same coordinates for all raws
                            for j in range(3):
                                sp_dum[:,j]=center[j]

                            # calculation of distance between spindle and all the surface points
                            dif_pow=(vlist_np_fl_edit[:,0:3]-sp_dum)**2
                            dif=dif_pow.sum(axis=1)
                            length=np.sqrt(dif)


                            #index of surface point which is nearest to the spindle
                            index_min=np.argmin(length)

                            xyz_min=vlist_np_fl_edit[index_min,0:3]
                            norm_min=vlist_np_fl_edit[index_min,3:6]
                            result.iloc[i,4:7]=xyz_min

                            result.iloc[i,7:10]=norm_min
                            result.iloc[i,10]=length[index_min]

                            #calculation of angle between normal vector and spindle vector
                            costheta=np.dot(unit_vector,norm_min);
                            theta=np.arccos(costheta);
                            theta=np.rad2deg(theta) # to degree

                            #calculation of angle between normal vector and a vector connected between spindle and the surface point
                            sp_suf_unit_vector=(xyz_min[:]-center[:])/np.linalg.norm(xyz_min[:]-center[:]) # unit vector from spindle to surface point
                            costheta2=np.dot(sp_suf_unit_vector,norm_min);
                            theta2=np.arccos(costheta2);
                            theta2=np.rad2deg(theta2) # to degree

                            #print theta
                            if theta>90:
                                theta=180-theta;
                                #print "flipped"
                            result.iloc[i,11]=theta

                            if theta2>90:
                                theta2=180-theta2;
                                print ("flipped")
                            result.iloc[i,12]=theta2
                            #print theta
                            result.iloc[i,0]=i;
                            result.iloc[i,1]=center[0];
                            result.iloc[i,2]=center[1];
                            result.iloc[i,3]=center[2];
                            result.iloc[i,13]=time_num
                            result.iloc[i,14]=spindle_name

                        #print result

                        #header = '{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}'.format('# of sp','sp_x','sp_y','sp_z','x_surf','y_surf','z_surf','nx','ny','nz','distance(pix)','theta(degree)','theta_check(degree)','time','spindle_name')
                        result.columns = ['# of sp','sp_x','sp_y','sp_z','x_surf','y_surf','z_surf','nx','ny','nz','distance(pix)','theta(degree)','theta_check(degree)','time','spindle_name']
                        filename = "result_"+bpsc_name+".txt"
                        #np.savetxt(filename,result,delimiter=",",fmt='%.3f',header=header,comments='')
                        result.to_csv(filename, sep=",", index=False, header=True)
                        
                        plt.figure(figsize=(8,8))
                        #plt.tight_layout()
                        plt.plot([-1,10000], [45, 45], 'r')
                        x=range(num_spindle)
                        y=result.iloc[:,11]
                        plt.scatter(range(num_spindle),result.iloc[:,11],s=100)
                        plt.xlim([-1,num_spindle])
                        plt.ylim([0,90])
                        plt.xlabel("# of spindle", fontsize=18)
                        plt.ylabel("Angle (degree) between \n surface normal and direction of spindle", fontsize=18)

                        filename = "result_"+bpsc_name+".pdf"
                        plt.savefig(filename)

In [8]:
# Add center of mass of spindles and their nearest-neighbor surface points to imx file "spindle_surface.imx" to confirm the analysis on Imaris
from lxml import etree
filelist=glob.glob('*00.imx')
tree = parse(filelist[0])
root = tree.getroot()
cpname = filelist[0]+"_edit.imx"
shutil.copyfile(filelist[0], cpname)
cptree = parse(cpname)
cproot = cptree.getroot()
output ="spindle_surface.imx"
output_xml ="spindle_surface_xml.xml"

In [9]:
# Add center of mass of spindles and their nearest-neighbor surface points to imx file "spindle_surface.imx" to confirm the analysis on Imaris
for bpsa in cproot.findall("bpSurfaceApplication"):
    for bpcg in bpsa.findall("bpComponentGroup"):
        #print bpcg.tag
        for bpsc in bpcg.findall("bpSurfaceComponent"):
            #print bpsc.tag
            for bpcg2 in bpsc.findall("bpComponentGroup"):
                #print bpcg2.tag
                added_bpcg=SubElement(bpsc,"bpComponentGroup")
                
                for bpsc2 in bpcg2.findall("bpSurfaceComponent"):
                    #print bpsc2.tag
                    added_bpsc=SubElement(added_bpcg,"bpSurfaceComponent")
                    
                    for name in bpsc2.findall("name"): #Folder, Frame131-140 etc.
                        bpsc_name = name.text
                        #print (bpsc_name)
                        added_name="added_"+bpsc_name
                        #print (name.text)
                        filename ="result_"+bpsc_name+".txt"
                        addpoint_xyz=np.loadtxt(filename,usecols=(1,2,3,4,5,6),delimiter=",")
                        xyz_py = addpoint_xyz.tolist()
                        
                        if isinstance(xyz_py[0], list)==False:
                            xyz_py = [xyz_py]                      
                        
                        #print (xyz_py)
                        
                        added_framename=SubElement(added_bpsc,"name")
                        added_framename.text=added_name #Folder, added_Frame131-140 etc.
                        
                        added_idnum=SubElement(added_bpsc,"id")
                        
                        for name2 in bpsc2.findall("id"): #Folder, Frame131-140 etc.
                            added_idnum.text=str(name2.text)
                        
                        added_ishidden=SubElement(added_bpsc,"isHidden")
                        added_ishidden.text="true"
                        
                        added_isnode=SubElement(added_bpsc,"isNode")
                        added_isnode.text="true"
                        
                        added_isexpanded=SubElement(added_bpsc,"isExpanded")
                        added_isexpanded.text="false"
                        
                            
                        added_material=SubElement(added_bpsc,"material")
                        
                        added_ac=SubElement(added_material,"ambientColor")
                        added_ac.text="((0.200,0.200,0.200))"
                        
                        added_dc=SubElement(added_material,"diffuseColor")
                        added_dc.text="((0.800,0.800,0.800))"

                        added_sc=SubElement(added_material,"specularColor")
                        added_sc.text="((0.000,0.000,0.000))"

                        added_ec=SubElement(added_material,"emissiveColor")
                        added_ec.text="((0.000,0.000,0.000))"

                        added_s=SubElement(added_material,"shininess")
                        added_s.text="(0.200)"

                        added_t=SubElement(added_material,"transparency")
                        added_t.text="(0.000)"
                        
                        
                        
                        
                        
                        
                        
                        
                        #print xyz_py
                        #counting # of spindles in one Frame
                        i=0
                        
                        for bpmp in bpsc2.findall("bpMeasurementPoints"):
                            #print(i)
                            added_bpmp=SubElement(added_bpsc,"bpMeasurementPoints")
                            for bpsc3 in bpmp.findall("bpSurfaceComponent"):
                                
                                added_bpsc2=SubElement(added_bpmp,"bpSurfaceComponent")
                                
                                added_mpname=SubElement(added_bpsc2,"name")
                                
                                for name2 in bpsc3.findall("name"):
                                    #print (name2.text)
                                    
                                    addpoint = "added_"+str(name2.text)
                                    added_mpname.text=addpoint
                                    
                                added_id=SubElement(added_bpsc2,"id")
                                for name2 in bpsc3.findall("id"):
                                    #print (name2.text)
                                    id_large=int(name2.text)+100
                                    added_id.text=str(id_large)
                                    
                                added_isHidden=SubElement(added_bpsc2,"isHidden")
                                for name2 in bpsc3.findall("isHidden"):
                                    #print (name2.text)
                                    added_isHidden.text=str(name2.text)
                                    
                                added_isNode=SubElement(added_bpsc2,"isNode")
                                for name2 in bpsc3.findall("isNode"):
                                    #print (name2.text)
                                    added_isNode.text=str(name2.text)
                                    
                                added_isExpanded=SubElement(added_bpsc2,"isExpanded")
                                for name2 in bpsc3.findall("isExpanded"):
                                    #print (name2.text)
                                    added_isExpanded.text=str(name2.text)
                                    
                                added_material=SubElement(added_bpsc2,"material")
                                added_ac=SubElement(added_material,"ambientColor")
                                added_ac.text="((0.200,0.200,0.200))"
                                
                                added_dc=SubElement(added_material,"diffuseColor")
                                added_dc.text="((1.000,0.000,0.000))"
                                
                                added_sc=SubElement(added_material,"specularColor")
                                added_sc.text="((0.000,0.000,0.000))"
                                
                                added_ec=SubElement(added_material,"emissiveColor")
                                added_ec.text="((0.000,0.000,0.000))"
                                
                                added_s=SubElement(added_material,"shininess")
                                added_s.text="(0.200)"
                                
                                added_t=SubElement(added_material,"transparency")
                                added_t.text="(0.000)"
                                
                            j=0

                            for point in bpmp.findall("Point"):

                                #print (point.get("Position"))
                                ti=point.get("TimeIndex")
                                added_point=SubElement(added_bpmp,"Point")

                                #print(i)
                                if j==0:
                                    #print("j=0")
                                    xyz_spindle=str(xyz_py[i][0])+" "+ str(xyz_py[i][1])+ " "+str(xyz_py[i][2])
                                    #print (xyz_spindle)
                                    added_point.set("Position",xyz_spindle)
                                    added_point.set("Name",'center')
                                    added_point.set("TimeIndex",str(ti))
                                    #cptree.write(output)

                                if j==1:
                                    #print("j=1")
                                    xyz_surface=str(xyz_py[i][3])+" "+ str(xyz_py[i][4])+ " "+str(xyz_py[i][5])
                                    #print (xyz_surface)
                                    added_point.set("Position",xyz_surface)
                                    added_point.set("Name",'surface_point')
                                    added_point.set("TimeIndex",str(ti))
                                    #point.get('Position') = xyz_surface
                                    #cptree.write(output)

                                j+=1
                                                    
                            added_ps=SubElement(added_bpmp,"PointShape")
                            added_ps.set("Shape","Sphere")
                            added_ps.set("Filled","true")
                                                    
                            added_pd=SubElement(added_bpmp,"PointDiameter")
                            added_pd.text="2.000"
                                                    
                            added_pl=SubElement(added_bpmp,"PointLabels")
                            added_pl.set("Name","false")
                            added_pl.set("Intensity","false")
                            added_pl.set("Angle","false")
                                                    
                                                    
                            added_lm=SubElement(added_bpmp,"LineMode")
                            added_lm.text="Pairs"
                                                    
                            added_v=SubElement(added_bpmp,"VisibleForAllTimepoints")
                            added_v.text="false"
                                                    
                            added_lw=SubElement(added_bpmp,"LineWidth")
                            added_lw.text="2"
                                                    
                            added_ll=SubElement(added_bpmp,"LineLabels")
                            added_ll.set("Distance","false")
                            added_ll.set("Velocity","false")
                                                    
                            added_lc=SubElement(added_bpmp,"LabelsColor")
                            added_lc.text="1.00000000 1.00000000 1.00000000"
                                                    
                            added_lf=SubElement(added_bpmp,"LabelsFont")
                            added_lf.set("Name","Arial")
                            added_lf.set("Size","12")
                            added_lf.set("Bold","false")
                            added_lf.set("Italic","false")
                            added_lf.set("Underline","false")
                                                    
                            added_am=SubElement(added_bpmp,"AutodepthMode")
                            added_am.set("Intersect","SolidObjectsSurface")
                            
                            
                            
                                
                                    
                            i+=1
                        nlist = i;
                        #print ("# of bpMeasurementPoints tag")
                        #print (nlist)
                        
#file = tostring(cptree,pretty_print="true")
cptree.write(output)
cptree.write(output_xml)